## Import dependencies

In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

## Load Data

### Read raw data

In [2]:
# Load raw data
df = pd.read_csv("../data/WA_Fn-UseC_-Telco-Customer-Churn.csv")

# Read first five rows of dataframe
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


## Data Cleaning

### Check for missing data

In [ ]:
df.isna().sum()

### Convert 'TotalCharges' and 'MonthlyCharges' to float

In [ ]:
df['TotalCharges'] = df['TotalCharges'].apply(lambda x: 0.0 if x == ' ' else float(x))

## Feature Engineering

### One-Hot Encode Features

In [ ]:
# Encode binary features
df['gender_encoded'] = df['gender'].apply(lambda x: 0 if x == "Female" else 1)
df['Partner_encoded'] = df['Partner'].apply(lambda x: 0 if x == "No" else 1)
df['Dependents_encoded'] = df['Dependents'].apply(lambda x: 0 if x == "No" else 1)
df['PhoneService_encoded'] = df['PhoneService'].apply(lambda x: 0 if x == "No" else 1)
df['PaperlessBilling_encoded'] = df['PaperlessBilling'].apply(lambda x: 0 if x == "No" else 1)
df['Churn_encoded'] = df['Churn'].apply(lambda x: 0 if x == "No" else 1)

### Label Encode Features

In [ ]:
from sklearn.preprocessing import LabelEncoder
pd.set_option("display.max_columns", None)

features_to_label_encode = ['MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 
                            'DeviceProtection',  'TechSupport', 'StreamingTV', 'StreamingMovies',
                             'Contract', 'PaymentMethod',]

def apply_label_encoding(df, list_features):
    le = LabelEncoder()

    for feature in list_features:
        df[f'{feature}_encoded'] = le.fit_transform(df[feature])
    
    return df

apply_label_encoding(df, features_to_label_encode)

## Feature Selection 

In [ ]:
# Recreate dataframe with
df_features = df[['Churn_encoded', 'gender_encoded','SeniorCitizen', 'Partner_encoded', 'Dependents_encoded',
                  'tenure', 'PhoneService_encoded', 'MultipleLines_encoded', 'InternetService_encoded',
                  'OnlineSecurity_encoded', 'OnlineBackup_encoded', 'DeviceProtection_encoded', 'TechSupport_encoded',
                  'StreamingTV_encoded', 'StreamingMovies_encoded', 'Contract_encoded','PaperlessBilling_encoded',
                  'PaymentMethod_encoded','MonthlyCharges', 'TotalCharges',
                  ]].copy()

In [ ]:
# Reset the index to ensure sequential indexing
df_features = df_features.reset_index(drop=True)

plt.figure(figsize=(12, 10))
sns.heatmap(df_features.corr(), 
            annot=True,
            cmap="Blues",
            fmt=".2f",
            linewidths=.5,
            square=True,
            cbar_kws={"shrink": .8}
        )

# Add labels, title, and adjust axes params
plt.title("Heat map of correlation matrix", fontsize=13, weight="bold")
plt.xlabel('Feature', fontsize=10)
plt.ylabel('Feature', fontsize=10)
plt.xticks(fontsize=8, rotation=45, ha='right')
plt.yticks(fontsize=8)
plt.tight_layout()
plt.show()

In [ ]:
# Less important features
less_important_features = ['gender_encoded', 'PhoneService_encoded', 'MultipleLines_encoded', 
                           'InternetService_encoded','StreamingTV_encoded', 'StreamingMovies_encoded',]

# Important features
selected_final_df = ['Churn_encoded', 'SeniorCitizen', 'Partner_encoded',
                                'Dependents_encoded', 'tenure', 'OnlineSecurity_encoded', 'OnlineBackup_encoded',
                                'DeviceProtection_encoded', 'TechSupport_encoded','Contract_encoded',
                                'PaperlessBilling_encoded', 'PaymentMethod_encoded', 'MonthlyCharges',
                                'TotalCharges']



## Cutting not important features

In [ ]:
final_df = df_features.drop(less_important_features, axis=1)

# final_df = df_features.copy()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf

### Preparing data

In [ ]:
X = final_df.drop(columns=["Churn_encoded"])
y = final_df["Churn_encoded"]

In [ ]:
# Séparation train/val/test (80/20 puis 20% de train pour val)
X_train_0, X_test, y_train_0, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# On prend 20% de X_train pour validation
X_train, X_val, y_train, y_val = train_test_split(
    X_train_0, y_train_0, test_size=0.2, random_state=42, stratify=y_train_0)

### Class weight for Churn

In [ ]:
# Add class weights for target
classes = np.unique(y)
weights = compute_class_weight(class_weight="balanced", classes=classes, y=y_train_0)
class_weights = dict(zip(classes, weights))

### Model definition

In [ ]:
def build_model():
    # Réseau avec 2 couches cachées de 64 neurones chacune
    # et une couche de sortie avec activation sigmoid pour classification
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(X_train.shape[1],)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    # Définition de la fonction de perte, de l'optimiseur et des métriques
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    model.summary()

    return model

In [ ]:
model = build_model()

### Training

In [ ]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=60,
    batch_size=128,
    class_weight=class_weights,
    verbose=1
)

In [ ]:
history.history

### Evaluation

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f"\nAccuracy on test set : {test_acc:.4f}")

In [ ]:
y_pred_probs = model.predict(X_test)

probs = y_pred_probs.ravel()

y_pred = (probs >= 0.7)
print("\nClassification Report :")
print(classification_report(y_test, y_pred))
y_test.shape, y_pred_probs.shape

In [ ]:

print("min, max, mean:", probs.min(), probs.max(), probs.mean())


### Confusion matrix

In [ ]:
print("Confusion matrix :")
cm = confusion_matrix(y_test, y_pred)
print(cm)

### Learning curves vizualisation

In [ ]:
def plot_loss_acc(history, validation=True):
    """
    Trace la loss et l'accuracy du modèle pendant l'entraînement.
    """
    plt.figure(figsize=(12, 4))

    # Loss
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Train Loss')
    if validation and 'val_loss' in history.history:
        plt.plot(history.history['val_loss'], label='Val Loss')
    plt.title('Évolution de la Loss')
    plt.xlabel('Époque')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid()

    # Accuracy
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Train Acc')
    if validation and 'val_accuracy' in history.history:
        plt.plot(history.history['val_accuracy'], label='Val Acc')
    plt.title("Évolution de l'Accuracy")
    plt.xlabel('Époque')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid()

    plt.tight_layout()
    plt.show()
    
plot_loss_acc(history)